# Week 3 - Denoising Filter Integration and Demonstration

**Goal:** Demonstrate the use of various denoising filters (Gaussian, Median, Bilateral, Box) that can be selected at runtime, similar to how they might be integrated into a frame processing loop. This notebook uses the functions defined in `object_sorter_model.preprocessing.filtering`.

**Key Concepts:**
- Different filters excel at handling different types of noise and have varying effects on image details and edges.
- A configurable system allows choosing the most appropriate filter for the current conditions or specific processing stage.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time # For simple timing demonstration

# --- Add project root to Python path for module imports ---
# This assumes the notebook is in 'object_sorter_model/notebooks/'
# and the package 'object_sorter_model' is one level up.
CURRENT_DIR = os.getcwd()
if os.path.basename(CURRENT_DIR) == 'notebooks':
    PROJECT_ROOT = os.path.abspath(os.path.join(CURRENT_DIR, '..'))
else: # Fallback if running from project root or elsewhere
    PROJECT_ROOT = CURRENT_DIR # Or specify absolute path

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
    print(f"Added to sys.path: {PROJECT_ROOT}")

try:
    from object_sorter_model.preprocessing import filtering # This is our refactored module
    # We might also want to load config if parameters come from there
    # from object_sorter_model.utils import config_loader 
except ImportError as e:
    print(f"Error importing filtering module: {e}")
    print("Make sure the project structure is correct and __init__.py files are present.")
    raise

# --- Configuration for the Notebook ---
OUTPUT_DIR_NOTEBOOK = "output_week3_notebook_denoising"
os.makedirs(OUTPUT_DIR_NOTEBOOK, exist_ok=True)

# Test image path (relative to project root)
# Ensure you have a suitable test image, e.g., lena.png or your own
# TEST_IMAGE_PATH = os.path.join(PROJECT_ROOT, "data", "sample_images", "esp32_test_image.jpg") 
# Using a common sample for better visual results with filters:
try:
    TEST_IMAGE_PATH = cv2.samples.findFile('lena.png')
    if not os.path.exists(TEST_IMAGE_PATH): # Fallback if samples.findFile fails but path exists
        TEST_IMAGE_PATH = os.path.join(PROJECT_ROOT, "data", "sample_images", "lena.png") # Assuming you add it
        if not os.path.exists(TEST_IMAGE_PATH):
             TEST_IMAGE_PATH = os.path.join(PROJECT_ROOT, "data", "sample_images", "esp32_test_image.jpg") # Final fallback
except:
    TEST_IMAGE_PATH = os.path.join(PROJECT_ROOT, "data", "sample_images", "esp32_test_image.jpg")


print("Setup complete. Filtering module imported.")
print(f"Using test image: {TEST_IMAGE_PATH}")

## 1. Helper Functions (Noise Addition & Display)
We'll use helper functions to add noise to our test image and to display multiple images for comparison.

In [ ]:
def add_noise_to_image(image, noise_type='gaussian', amount=0.05, salt_pepper_ratio=0.5):
    """Adds specified noise to an image. (Adapted from previous scripts)"""
    noisy_image = image.copy()
    if noise_type == "gaussian":
        row, col, ch = image.shape
        sigma = amount * 255
        gauss = np.random.normal(0, sigma, (row, col, ch))
        noisy_image = np.clip(image.astype(np.float32) + gauss, 0, 255).astype(np.uint8)
    elif noise_type == "salt_pepper":
        row, col, ch = image.shape
        s_vs_p = salt_pepper_ratio
        num_pixels_to_affect = int(amount * row * col)
        
        num_salt = int(num_pixels_to_affect * s_vs_p)
        salt_coords_y = np.random.randint(0, row - 1, num_salt)
        salt_coords_x = np.random.randint(0, col - 1, num_salt)
        for i in range(ch): noisy_image[salt_coords_y, salt_coords_x, i] = 255
            
        num_pepper = int(num_pixels_to_affect * (1. - s_vs_p))
        pepper_coords_y = np.random.randint(0, row - 1, num_pepper)
        pepper_coords_x = np.random.randint(0, col - 1, num_pepper)
        for i in range(ch): noisy_image[pepper_coords_y, pepper_coords_x, i] = 0
    return noisy_image

def display_multiple_images(img_dict, main_title="Image Comparison", cols=2, save_base_path=None):
    num_images = len(img_dict)
    if num_images == 0: return
    rows = int(np.ceil(num_images / cols))
    fig_width = cols * 5
    fig_height = rows * 5 + (0.5 if main_title else 0)

    plt.figure(figsize=(fig_width, fig_height))
    if main_title: plt.suptitle(main_title, fontsize=16)
    
    filenames_map = {}
    for i, (title, img_bgr) in enumerate(img_dict.items()):
        plt.subplot(rows, cols, i + 1)
        if img_bgr is None:
            plt.title(title + "\n(N/A)")
            plt.axis('off')
            continue
        
        if img_bgr.ndim == 3: # Color
            plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
        else: # Grayscale
            plt.imshow(img_bgr, cmap='gray')
        plt.title(title)
        plt.axis('off')

        if save_base_path:
            filename_part = title.lower().replace(" ", "_").replace("(", "").replace(")", "").replace("=", "").replace(",", "").replace("\n", "_").replace(":", "")
            if len(filename_part) > 40: filename_part = filename_part[:40]
            full_save_path = os.path.join(OUTPUT_DIR_NOTEBOOK, f"{save_base_path}_{i}_{filename_part}.png")
            cv2.imwrite(full_save_path, img_bgr)
            filenames_map[title] = full_save_path
            
    plt.tight_layout(rect=[0, 0, 1, 0.95 if main_title else 1])
    plt.show()
    if save_base_path:
        print("Saved images:")
        for title, path in filenames_map.items(): print(f"- '{title}': {path}")

print("Helper functions for noise and display defined.")

## 2. Load Test Image and Add Noise
First, we load our test image and create noisy versions to demonstrate the filters.

In [ ]:
img_original_bgr = cv2.imread(TEST_IMAGE_PATH)
if img_original_bgr is None:
    print(f"FATAL: Test image not found at '{TEST_IMAGE_PATH}'. Please check the path.")
    # Create a dummy if absolutely necessary, but it's better to have a real image
    img_original_bgr = np.random.randint(0, 256, (256, 256, 3), dtype=np.uint8)
    cv2.putText(img_original_bgr, "Dummy Image", (50,128), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)


# Create noisy versions
img_gaussian_noisy = add_noise_to_image(img_original_bgr, noise_type='gaussian', amount=0.08)
img_sp_noisy = add_noise_to_image(img_original_bgr, noise_type='salt_pepper', amount=0.05)

images_to_show_initial = {
    "Original": img_original_bgr,
    "Gaussian Noise (Amount=0.08)": img_gaussian_noisy,
    "Salt & Pepper Noise (Amount=0.05)": img_sp_noisy
}
display_multiple_images(images_to_show_initial, "Original and Noisy Images", cols=3, save_base_path="01_initial")

## 3. Demonstrating Filter Selection and Application

We will use the `filtering.apply_denoising_filter` function from our `object_sorter_model` package. This function acts as a switch to select and apply different filters based on input parameters.

We will test each filter type on appropriately noised images.

In [ ]:
# --- Test Gaussian Filter ---
filter_type_gauss = 'gaussian'
params_gauss = {'kernel_size_wh': (7, 7), 'sigma_x': 2.0} # Example parameters
print(f"\nApplying {filter_type_gauss} filter with params: {params_gauss}")
start_time = time.time()
img_filtered_gauss = filtering.apply_denoising_filter(img_gaussian_noisy, filter_type_gauss, params_gauss)
print(f"  Processing time: {time.time() - start_time:.4f}s")

# --- Test Median Filter ---
filter_type_median = 'median'
params_median = {'kernel_size': 7} # Example parameters
print(f"\nApplying {filter_type_median} filter with params: {params_median}")
start_time = time.time()
img_filtered_median = filtering.apply_denoising_filter(img_sp_noisy, filter_type_median, params_median)
print(f"  Processing time: {time.time() - start_time:.4f}s")

# --- Test Bilateral Filter ---
filter_type_bilateral = 'bilateral'
params_bilateral = {'d': 9, 'sigma_color': 75, 'sigma_space': 75} # Example parameters
print(f"\nApplying {filter_type_bilateral} filter with params: {params_bilateral}")
start_time = time.time()
# Bilateral is often best on images with complex textures but less extreme noise, or for smoothing while preserving edges.
# Let's apply it to the Gaussian noisy image for this general demo.
img_filtered_bilateral = filtering.apply_denoising_filter(img_gaussian_noisy, filter_type_bilateral, params_bilateral)
print(f"  Processing time: {time.time() - start_time:.4f}s")

# --- Test Box Filter ---
filter_type_box = 'box'
params_box = {'kernel_size_wh': (5,5)} # Example parameters
print(f"\nApplying {filter_type_box} filter with params: {params_box}")
start_time = time.time()
img_filtered_box = filtering.apply_denoising_filter(img_gaussian_noisy, filter_type_box, params_box)
print(f"  Processing time: {time.time() - start_time:.4f}s")


# --- Display Results ---
results_gaussian = {
    "Gaussian Noisy Input": img_gaussian_noisy,
    f"{filter_type_gauss.capitalize()} Filtered": img_filtered_gauss
}
display_multiple_images(results_gaussian, "Gaussian Filter Demonstration", cols=2, save_base_path="02_gaussian")

results_median = {
    "Salt & Pepper Noisy Input": img_sp_noisy,
    f"{filter_type_median.capitalize()} Filtered": img_filtered_median
}
display_multiple_images(results_median, "Median Filter Demonstration", cols=2, save_base_path="03_median")

results_bilateral_box = {
    "Gaussian Noisy Input (for Bilateral/Box)": img_gaussian_noisy,
    f"{filter_type_bilateral.capitalize()} Filtered": img_filtered_bilateral,
    f"{filter_type_box.capitalize()} Filtered": img_filtered_box
}
display_multiple_images(results_bilateral_box, "Bilateral and Box Filter Demonstration", cols=3, save_base_path="04_bilateral_box")

## 4. Comparison of Filters on a Single Noisy Image

Let's apply all filters to the same Gaussian noisy image to compare their effects side-by-side.

In [ ]:
comparison_input_img = img_gaussian_noisy # Use the Gaussian noisy image

filtered_gaussian_comp = filtering.apply_denoising_filter(comparison_input_img, 'gaussian', {'kernel_size_wh': (5,5), 'sigma_x': 1.5})
filtered_median_comp = filtering.apply_denoising_filter(comparison_input_img, 'median', {'kernel_size': 5})
filtered_bilateral_comp = filtering.apply_denoising_filter(comparison_input_img, 'bilateral', {'d': 9, 'sigma_color': 75, 'sigma_space': 75})
filtered_box_comp = filtering.apply_denoising_filter(comparison_input_img, 'box', {'kernel_size_wh': (5,5)})

all_filters_comparison = {
    "Original": img_original_bgr,
    "Gaussian Noisy Input": comparison_input_img,
    "Gaussian Filtered": filtered_gaussian_comp,
    "Median Filtered": filtered_median_comp,
    "Bilateral Filtered": filtered_bilateral_comp,
    "Box Filtered": filtered_box_comp
}
display_multiple_images(all_filters_comparison, "Comparison of Denoising Filters on Gaussian Noise", cols=3, save_base_path="05_all_filters_comparison")

## 5. Observations and How to Use in Project Pipeline

- **Gaussian Filter:** Effective for general smoothing and reducing Gaussian-like noise. Tends to blur edges. Parameters: `kernel_size_wh`, `sigma_x`.
- **Median Filter:** Excellent for removing salt-and-pepper (impulse) noise. Preserves edges better than Gaussian filter. Parameter: `kernel_size`.
- **Bilateral Filter:** Smooths images while preserving edges. Good for reducing noise in textured areas without losing too much detail. More computationally intensive. Parameters: `d` (neighborhood diameter), `sigma_color`, `sigma_space`.
- **Box Filter (Mean Filter):** Simple averaging filter. Causes significant blurring, including edges. Parameter: `kernel_size_wh`.

**Integration into `run_pipeline_test.py`:**

The `filtering.apply_denoising_filter(frame, filter_type, params)` function is designed to be called within your main pipeline script (`run_pipeline_test.py`).

1.  **Configuration:** You would add a "denoising_filter" section to your `camera_params.json` (or a more general `pipeline_settings.json`):
    ```json
    // In camera_params.json or similar
    "preprocessing_params": {
        // ... other params ...
        "denoising_filter": {
            "apply": true, // or false
            "type": "bilateral", // "gaussian", "median", "box", "none"
            "params": { // Parameters specific to the chosen type
                "d": 9, 
                "sigma_color": 50, 
                "sigma_space": 50 
                // For Gaussian: "kernel_size_wh": [5,5], "sigma_x": 1.5
                // For Median: "kernel_size": 5
                // For Box: "kernel_size_wh": [3,3]
            }
        }
    }
    ```

2.  **In `run_pipeline_test.py` (Conceptual):**
    ```python
    # ... after geometric corrections and other preprocessing ...
    
    # Load denoising config
    # main_config = config_loader.load_json_config(main_config_filepath) # Already loaded
    preprocessing_cfg = main_config.get("preprocessing_params", {})
    denoising_config = preprocessing_cfg.get("denoising_filter", {})
    apply_denoising = denoising_config.get("apply", False)

    if apply_denoising:
        denoise_type = denoising_config.get("type", "none")
        denoise_params = denoising_config.get("params", {})
        
        print(f"Applying denoising filter: {denoise_type} with params: {denoise_params}")
        current_processed_image = filtering.apply_denoising_filter(
            current_processed_image, 
            denoise_type, 
            denoise_params
        )
        if current_processed_image is not None:
            cv2.imwrite(os.path.join(output_dir, f"XX_denoised_{denoise_type}.png"), current_processed_image)
        else:
            print(f"Denoising filter '{denoise_type}' returned None. Skipping further processing with it.")
            # Potentially revert to image before this step if needed for pipeline continuation
    
    # ... continue with next pipeline stage (e.g., object detection) ...
    ```
This notebook demonstrates how the choice of filter can be made externally (e.g., via configuration) and applied to an image using the centralized `apply_denoising_filter` function.